In [ ]:
import tensorflow as tf
from transformer.dataset import get_dataset, preprocess_sentence
from transformer import model
from sentence_transformers import SentenceTransformer
import datetime
import numpy as np
from pymilvus import connections, utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection


class MilvusCircuitStorage:
    NEAREST_NEIGHBORS = 3
    def __init__(self, collection_id=1):
        self.collection_id = collection_id
        connections.connect("default", host="xxx", port="xxxx")
        self._create_collection()
    
    def show_all_vectors(self):
        results = self.collection.query(expr="id>=0",output_fields=["embedding", "id"])
        vectors = [item["embedding"] for item in results]
        ids = [item["id"] for item in results]
        # Convert vectors to a numpy array
        vectors = np.array(vectors)
        print("Vector shape:", vectors.shape)
        print(vectors)


    def _create_collection(self):
        DIMENSIONS = 384
        COLLECTION_NAME = "circuit_embeddings" + str(self.collection_id)
        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSIONS),
            FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=255)
        ]
        self.schema = CollectionSchema(fields, description="Vector collection")
        if utility.has_collection(COLLECTION_NAME):
           utility.drop_collection(COLLECTION_NAME)

        self.collection = Collection(name=COLLECTION_NAME, schema=self.schema)
        self._create_index()
        self._load_in_memory()

    def insert_vector(self, texts, vectors):
        # print(texts, vectors)
        self.collection.insert([vectors[0], texts])

    def _load_in_memory(self):
        self.collection.load()

    def _create_index(self):
        index_params = {
            "metric_type": "COSINE",
            "index_type": "IVF_FLAT",
            "params": {"nlist": 128}
        }
        self.collection.create_index(field_name="embedding", index_params=index_params)
    
    def search_n_nearest(self, query_vector, query_text):
        search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}
        results = self.collection.search(
            data=query_vector,
            anns_field="embedding",
            param=search_params,
            limit=MilvusCircuitStorage.NEAREST_NEIGHBORS,
            output_fields=["text"]
        )

        outputs = []
        print(results)
        # print("****")
        for hit in results[0]:
            if hit.entity.get("text") != query_text:
                outputs.append(hit.entity.get("text"))
        return outputs


    def delete(self, key):
        self.collection.delete(f"id == {key}")
        self.collection.flush()
    
    def close(self):
        connections.disconnect("default")


class SemanticMemory:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.vector_db = MilvusCircuitStorage()
        self.model = SentenceTransformer(model_name)
        self.conversation_history = []
        self.vector_dimension = 384
        
    def add_interaction(self, user_query, bot_response):
        interaction = {
            'question': user_query,
            'answer': bot_response,
            'timestamp': datetime.datetime.now()
        }
        self.conversation_history.append(interaction)
        embedding = self.model.encode([user_query])[0]
        self.vector_db.insert_vector([user_query], [embedding.reshape(1, -1)])
    
    def last_conversation(self):
        return self.conversation_history[-1] if self.conversation_history else None

        
    def retrieve_relevant_memory(self, query):
        query_vector = self.model.encode([query])[0]
        texts = self.vector_db.search_n_nearest(query_vector.reshape(1, -1), query)
        return texts

    def __del__(self):
        print(f"Memory is being cleared.")
        self.vector_db.close()


# memory = SemanticMemory()
# memory.add_interaction("Hi, How are you?", "I am george soros")
# memory.add_interaction("What do you do ?", "I run hedge fund")
# memory.retrieve_relevant_memory("Hello")
# memory.retrieve_relevant_memory("How are you?")



/Users/bhargavcn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bhargavcn/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def is_valid_context(candidates):
  MAX_INPUT = 65
  return len(candidates) == 1 or (sum(len(x) for x in candidates) < MAX_INPUT)

def context_builder(contexts):
  text = ""
  for context in contexts:
    text += " " + context
  return text
    

from collections import deque
def context_setter(user_input, memory: SemanticMemory):
  context_candidates = deque([])
  last_question = memory.last_conversation()
  if not last_question:
    return ""
  for relevant in memory.retrieve_relevant_memory(user_input):
    context_candidates.append(relevant)
  context_candidates.append(last_question["question"])

  while(not is_valid_context(context_candidates)):
    context_candidates.popleft()
  return context_builder(context_candidates)


In [3]:
import numpy as np

def evaluate(hparams, chatbot, tokenizer):
    memory = SemanticMemory()
    while True:
        user_input = input("You: ")
        print("You:"+ user_input)
        if user_input.lower() == "quit":
            print("Exiting chatbot. Goodbye!")
            break
        inputval = context_setter(user_input, memory)
        inputval += user_input
        output = predict(hparams, chatbot, tokenizer, inputval)
        final_version = output
        for i in range(2):
            response = predict(hparams, chatbot, tokenizer, user_input+ "? "+ final_version + "Complete the full sentence.")
            print(final_version)
            if response != output:
                final_version += response + ". "
                output = response
                continue
            break
        
        # lastresponse = memory.last_conversation()
        # if lastresponse and lastresponse["answer"] == output:
        #     print("Running again...")
        #     output = predict(hparams, chatbot, tokenizer, user_input)
        
        print(f"Bot: {final_version}")
        memory.add_interaction(user_input, final_version)

In [4]:
def inference(hparams, chatbot, tokenizer, sentence):
    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(
        hparams.start_token + tokenizer.encode(sentence) + hparams.end_token, axis=0
    )

    output = tf.expand_dims(hparams.start_token, 0)

    for i in range(hparams.max_length):
        predictions = chatbot(inputs=[sentence, output], training=False)

        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # return the result if the predicted_id is equal to the end token
        if tf.equal(predicted_id, hparams.end_token[0]):
            break

        # concatenated the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)


def predict(hparams, chatbot, tokenizer, sentence):
    prediction = inference(hparams, chatbot, tokenizer, sentence)
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size]
    )
    return predicted_sentence

In [5]:

# import pickle
#     PREFIX = hparams.save_model+".data/"
#     pickle.dump(hparams, open(PREFIX+"main_config.pkl", "wb"), protocol=0)
    
#     file = open(PREFIX+"sorostokenizer.pkl", 'rb')    
#     tokenizer2 = pickle.load(file)
#     file = open("main_config.pkl", 'rb')    
#     CONFIG = pickle.load(file)
# python3 /home/chickmagalurnanjun.b/soros/train.py --save_model model.h5 --batch_size 80 --epochs 40 
# --max_samples 50000 --max_length 65 --num_heads 8 --d_model 512 --num_units 2048 --dropout 0.05 --num_layers 4
class HYPERPARAMS:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

        
hparams = HYPERPARAMS(max_samples= 50000, max_length=65 , num_heads=8, d_model=512, num_units=2048, dropout=0.05, num_layers=4, batch_size=80, vocab_size=8185, save_model="model.h5")
data, tokenizer = get_dataset(hparams)
chatbot = tf.keras.models.load_model(
    hparams.save_model,
    custom_objects={
        "PositionalEncoding": model.PositionalEncoding,
        "MultiHeadAttentionLayer": model.MultiHeadAttentionLayer,
    },
    compile=False,
)
cbot = chatbot


2025-04-17 05:34:46.360324: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-17 05:34:46.360586: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-17 05:34:46.360608: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-17 05:34:46.361122: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-17 05:34:46.361410: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


********* VOCAB_SIZE: 8151******************


In [6]:
hparams.max_length = 300
evaluate(hparams, chatbot, tokenizer)

You:With silver at about $30 per ounce, how does it compare to gold as an inflation hedge?
during reagangs benign circle the sharp competition from reality , the experiment is likely to be punctuated with such breaks until participants cease to fear them . that is where the second approach using my hypothesis about the credit cycle comes in handy .
during reagangs benign circle the sharp competition from reality , the experiment is likely to be punctuated with such breaks until participants cease to fear them . that is where the second approach using my hypothesis about the credit cycle comes in handy .the financial markets are inherently unstable stability can be maintained only if it is made an objective of public policy .. 
Bot: during reagangs benign circle the sharp competition from reality , the experiment is likely to be punctuated with such breaks until participants cease to fear them . that is where the second approach using my hypothesis about the credit cycle comes in handy 

In [ ]:
# evaluate(hparams, chatbot, tokenizer)

You:  Current Gold price $2,400 an ounce, would you recommend adding gold to my current portfolio at this price?


Bot: this would effectively meant a high exchange rate policy .


You:  With silver trading at about $30 per ounce, how does it compare to gold as an inflation hedge?


Bot: this would involve not only putting our own house in order but also bringing a new international financial order into existence which is appropriate to the altered circumstances .


You:  What is the altered circumstances ?


...

Bot: the decline in the japanese stock market has turned into a veritable rout in the last few days . the market as a whole has dropped by some but the stocks that i own , which are all real estate related , are off from their recent highs . the bond market ,


You:  Continue the sentence


Bot: the decline in the japanese stock market has turned into a veritable rout in the last few days . the market as a whole has dropped by some but the stocks that i own , which are all real estate related , are off from their recent highs . the bond market ,


You:  exit


Exiting chatbot. Goodbye!


In [16]:
hparams.max_length = 300
# evaluate(hparams, chatbot, tokenizer)
# !python3 -m pip install langchain==0.2.0

In [19]:
from langchain_core.language_models import LLM
# import torch

class MyTransformerQA(LLM):
    def __init__(self, *args, **kwargs):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop=None, run_manager = None, chatbot=None)-> str:
        # Assume simple Q&A structure: extract question and context
        # Format: "Context: ... \n Question: ..."
        if "Question:" not in prompt or "Context:" not in prompt:
            return "Invalid input format."
        print(prompt or "No Prompt")
        context = prompt.split("Context:")[1].split("Question:")[0].strip()
        question = prompt.split("Question:")[1].split("Assistant:")[0].strip()
        print(context)
        print(question)

        output = predict(hparams, cbot, tokenizer, context+question or "Hello")
        return output

In [20]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

qa_model = MyTransformerQA("deepset/bert-base-cased-squad2")

memory = ConversationBufferMemory()
prompt_template = """
        You are role of chatbot to provide investment insights in this conversation and will answer the related questions.
        Context: {history}
        Question: {input}
        Assistant: """
        
prompt = PromptTemplate(
input_variables=["history", "input"],
template=prompt_template
)

chat = ConversationChain(
    prompt=prompt,
    llm=qa_model,
    memory=memory,
    verbose=True
)

chat.run("Who are you?")



> Entering new ConversationChain chain...
Prompt after formatting:

        You are role of chatbot to provide investment insights in this conversation and will answer the related questions.
        Context: 
        Question: Who are you?
        Assistant: 

        You are role of chatbot to provide investment insights in this conversation and will answer the related questions.
        Context: 
        Question: Who are you?
        Assistant: 

Who are you?

> Finished chain.


'the fact is that the system survived and the recent changes in economic management improve the chances that it will continue to survive .'

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Choose a light model for CPU
model_name = "google/flan-t5-small"  # or "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def synthesize_response(a, context_history, user_question):
    prompt = f"""
    Breifly elaborate the following sentences:
    {user_question}
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=False).to("cpu")
    outputs = model.generate(**inputs, max_length=1000, num_beams=4, early_stopping=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
bot_out = "Paris is a beautiful city and a popular tourist destination."
vec_context = "i am now inclined to close out the positions i hold in nearby months and establish a short position for next spring. the price would be lower, but the discount per month would also be lower. if my analysis is correct, the later the break comes the bigger it will be the price would be lower, but the discount per month would also be lower. if my analysis is correct, the later the break comes the bigger it will be. the views i have outlined here are sufficient to serve as a basis for what i call macroeconomic investment decisions... the currencies would be a greater degree of stability than we have experienced since the breakdown of bretton woods. using the notation adopted earlier, we can depict the key relationship in the newly emerging state of affairs as follows the picture is much simpler than that of the imperial circle be exchange rates are stabilize."
vec_context = "China targets U.S. services and other areas as it decries 'meaningless' tariff hikes on goods"
question = "Trump tarrifs hit a new high"
context = "the bundesbank has been consistent in the pursuit of its objectives , especially if we include institutional self preservation among those objectives , and amazing successful."

final = synthesize_response(bot_out, vec_context, context)
print("Final Answer:", final)



def generate_question(text):
        prompt = f"""
          {text}
          Refine the titles as curious questions.
          """
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cpu")
        outputs = model.generate(**inputs, max_length=256, num_beams=4, early_stopping=True)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

Final Answer: Breifly says the bundesbank has been consistent in the pursuit of its objectives , especially if we include institutional self preservation among those objectives , and amazing successful


In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization")
def summarize_and_clean(text):
    return summarizer(text, max_length=500, min_length=30, do_sample=False)[0]['summary_text']

text = "using the concept of reflexivity to predict the future course of events is itself a re flexive process , and a high degree of success is often the precursor to a severe setback .using the concept of reflexivity to predict the future course of events is itself a re flexive process , and a high degree of success is often the precursor to a severe setback . it is clear from the experiment that my perceptions are partial and subject to correction by events ..i expect the break to be temporary there is no point in raising cash now . i have done so , i have had considerable difficulty in arriving at this conclusion . in the last few months of the s , the imperial circle was unwinding and there was a danger it would start spinning in."
print(summarize_and_clean(text))

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
Your max_length is set to 500, but your input_length is only 119. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)


 my brief account of recent events is sufficient to show that regulators are not exempt from bias . The collapse of the japanese bond market was the first part of the international economy was u . s .


In [64]:
import feedparser
import random

rss_url = "https://www.ft.com/rss/home"
feed = feedparser.parse(rss_url)

print(feed)
i = 3
chosen = []
while(i>0):
    c = random.choice(feed["entries"])
    if c in chosen:
        continue
    print(c["title"])
    print(generate_question(c["title"]))
    i-=1


{'bozo': False, 'entries': [{'title': 'ECB cuts rates to 2.25% amid Trump trade war', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.ft.com/rss/home', 'value': 'ECB cuts rates to 2.25% amid Trump trade war'}, 'summary': 'Growth and inflation set to slow as central bankers prepare for tariff impact', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://www.ft.com/rss/home', 'value': 'Growth and inflation set to slow as central bankers prepare for tariff impact'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.ft.com/content/70934d09-dfa5-4ac0-b5a2-2d7ae39c8d95'}], 'link': 'https://www.ft.com/content/70934d09-dfa5-4ac0-b5a2-2d7ae39c8d95', 'id': '70934d09-dfa5-4ac0-b5a2-2d7ae39c8d95', 'guidislink': False, 'published': 'Thu, 17 Apr 2025 14:06:31 GMT', 'published_parsed': time.struct_time(tm_year=2025, tm_mon=4, tm_mday=17, tm_hour=14, tm_min=6, tm_sec=31, tm_wday=3, tm_yday=107, tm_isdst=0), 'media_thumbnail': [